# Data Prep - Visitors
Geoff Pidcock | PacifImpact
## Scope
Model visitor data for the following countries...
- FJ, Cook Island, Soloman Islands - MUST HAVE
- Vanuatu, Samoa - NICE TO HAVE

Document Assumptions in markdown as we go... <br>

and Save to PostGres <br>
Using the following data model
<br>
![Using the following data model](../references/ais-hack-country_metrics_schema.png)

## Reference 
- [modelling/survey gsheet](https://docs.google.com/spreadsheets/d/1qKgOixdJtwYD0jB1ouN4-pUIqrEXEPJzAqWfYbIY06g/edit#gid=0&fvid=1204782497)

# Setup

In [1]:
# Setup
import os
from dotenv import load_dotenv, find_dotenv

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

DBUNAME = os.environ.get("DBUNAME")
DBPASSWORD = os.environ.get("DBPASSWORD")
DBHOST = os.environ.get("DBHOST")
DBPORT = os.environ.get("DBPORT")
DBNAME = os.environ.get("DBNAME")

import sqlalchemy
import psycopg2
import pandas as pd
import numpy as np

connection_str = 'postgresql+psycopg2://'+DBUNAME+':'+DBPASSWORD+'@'+DBHOST+':'+DBPORT+'/'+DBNAME

In [ ]:
# creating the table (only execute this once!)

# with engine.connect() as con:
#     con.execute("""
#     drop table if exists public.country_metrics;
#     create table public.country_metrics
#     (
#         metric_key text not null
#         ,frequency text not null
#         ,country text not null
#         ,category text not null
#         ,name text not null
#         ,date date not null
#         ,value float8 null
#         ,properties json null
#         ,primary key(metric_key)
#     );;
#     """)

# Fiji (FJ)
**Source:** [Stats Fiji, Visitor Arrival Stats (accessed September 2020)](https://www.statsfiji.gov.fj/index.php/statistics/tourism-and-migration-statistics/visitor-arrivals-statistics)

Notes on usage
- Downloaded the "Visitor Arrivals Tables" file and Referred to `Table 1`
- Using a June snapshot with numbers up to June 2020
- Monthly aggregation
- Numbers are not seasonally adjusted
- Further commentary can be found [here](https://www.statsfiji.gov.fj/index.php/latest-releases/tourism-and-migration/visitor-arrivals/1087-provisional-visitor-arrivals-july-2020)

In [3]:
# evaluate dataset 1 - visitor arrival table
# source - https://www.statsfiji.gov.fj/index.php/component/advlisting/?view=download&format=raw&fileId=2148
# this was quite manual
fj_data = pd.read_excel('../data/raw/fj/Visitor-Arrivals-Tables.xlsx'
                          ,sheet_name='T1'
                          ,usecols='B:F,H:J,L,N'
                          ,skiprows=15
                          ,nrows=32
                          ,header=None
                          ,names=['year','month','arrivals-resident','arrivals-visitor','arrivals-total',
                                 'departures-resident','departures-visitors','departures-total',
                                 'total-in-transit','passenger-movement'])
display(fj_data.head(10),fj_data.tail(10))

,year,month,arrivals-resident,arrivals-visitor,arrivals-total,departures-resident,departures-visitors,departures-total,total-in-transit,passenger-movement
0,2018.0,January,19324.0,62648.0,81972.0,15040.0,72627.0,87667.0,14484.0,184123.0
1,NaN,February,9650.0,48798.0,58448.0,12016.0,45491.0,57507.0,9385.0,125340.0
2,NaN,March,9676.0,60058.0,69734.0,12445.0,50698.0,63143.0,8304.0,141181.0
3,NaN,April,13739.0,63535.0,77274.0,13666.0,62345.0,76011.0,7699.0,160984.0
4,NaN,May,12975.0,67290.0,80265.0,12155.0,63679.0,75834.0,8210.0,164309.0
5,NaN,June,12681.0,81653.0,94334.0,13107.0,70851.0,83958.0,7813.0,186105.0
6,NaN,July,11502.0,95061.0,106563.0,14026.0,92072.0,106098.0,9648.0,222309.0
7,NaN,August,10418.0,88693.0,99111.0,14133.0,88552.0,102685.0,8007.0,209803.0
8,NaN,September,10198.0,81437.0,91635.0,13729.0,76513.0,90242.0,9700.0,191577.0
9,NaN,October,12667.0,79077.0,91744.0,12341.0,81194.0,93535.0,9200.0,194479.0


,year,month,arrivals-resident,arrivals-visitor,arrivals-total,departures-resident,departures-visitors,departures-total,total-in-transit,passenger-movement
22,NaN,October,10396.0,77467.0,87863.0,12610.0,80129.0,92739.0,9594.0,190196.0
23,NaN,November,9688.0,69123.0,78811.0,17822.0,68917.0,86739.0,7350.0,172900.0
24,NaN,December,18465.0,73740.0,92205.0,22026.0,61528.0,83554.0,13034.0,188793.0
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,2020.0,January,18238.0,65386.0,83624.0,15434.0,76195.0,91629.0,12571.0,187824.0
27,NaN,February,10448.0,46343.0,56791.0,11748.0,43748.0,55496.0,8366.0,120653.0
28,NaN,March,8166.0,27972.0,36138.0,7608.0,34782.0,42390.0,4064.0,82592.0
29,NaN,April,510.0,678.0,1188.0,532.0,1488.0,2020.0,28.0,3236.0
30,NaN,May,410.0,709.0,1119.0,259.0,940.0,1199.0,0.0,2318.0
31,NaN,June,628.0,413.0,1041.0,650.0,645.0,1295.0,0.0,2336.0


##### proposed prep steps
- remove nulls
- forward fill year
- cast year and month as date
- melt
- append with metadata
- check and cast types

In [6]:
# remove nulls (excepting year)
fj_data_prep = fj_data.copy()
fj_data_prep.dropna(axis=0,subset=['month'],inplace=True)
display(fj_data_prep.shape,fj_data.shape)
# output makes sense as there are two breaks for years

(30, 10)

(32, 10)

In [7]:
# forward fill year
fj_data_prep.loc[:,'year'] = fj_data_prep.loc[:,'year'].ffill()
# fj_data_prep #dev check

,year,month,arrivals-resident,arrivals-visitor,arrivals-total,departures-resident,departures-visitors,departures-total,total-in-transit,passenger-movement
0,2018.0,January,19324.0,62648.0,81972.0,15040.0,72627.0,87667.0,14484.0,184123.0
1,2018.0,February,9650.0,48798.0,58448.0,12016.0,45491.0,57507.0,9385.0,125340.0
2,2018.0,March,9676.0,60058.0,69734.0,12445.0,50698.0,63143.0,8304.0,141181.0
3,2018.0,April,13739.0,63535.0,77274.0,13666.0,62345.0,76011.0,7699.0,160984.0
4,2018.0,May,12975.0,67290.0,80265.0,12155.0,63679.0,75834.0,8210.0,164309.0
5,2018.0,June,12681.0,81653.0,94334.0,13107.0,70851.0,83958.0,7813.0,186105.0
6,2018.0,July,11502.0,95061.0,106563.0,14026.0,92072.0,106098.0,9648.0,222309.0
7,2018.0,August,10418.0,88693.0,99111.0,14133.0,88552.0,102685.0,8007.0,209803.0
8,2018.0,September,10198.0,81437.0,91635.0,13729.0,76513.0,90242.0,9700.0,191577.0
9,2018.0,October,12667.0,79077.0,91744.0,12341.0,81194.0,93535.0,9200.0,194479.0


In [12]:
# cast year and month as date
fj_data_prep['year'] = fj_data_prep['year'].astype('int')
fj_data_prep['year_month'] = fj_data_prep['year'].astype('str') +' '+ fj_data_prep['month']
fj_data_prep['date'] = pd.to_datetime(fj_data_prep['year_month'],format='%Y %B')
fj_data_prep.head()

,year,month,arrivals-resident,arrivals-visitor,arrivals-total,departures-resident,departures-visitors,departures-total,total-in-transit,passenger-movement,year_month,date
0,2018,January,19324.0,62648.0,81972.0,15040.0,72627.0,87667.0,14484.0,184123.0,2018 January,2018-01-01
1,2018,February,9650.0,48798.0,58448.0,12016.0,45491.0,57507.0,9385.0,125340.0,2018 February,2018-02-01
2,2018,March,9676.0,60058.0,69734.0,12445.0,50698.0,63143.0,8304.0,141181.0,2018 March,2018-03-01
3,2018,April,13739.0,63535.0,77274.0,13666.0,62345.0,76011.0,7699.0,160984.0,2018 April,2018-04-01
4,2018,May,12975.0,67290.0,80265.0,12155.0,63679.0,75834.0,8210.0,164309.0,2018 May,2018-05-01


In [14]:
# melt
fj_data_prep2 = fj_data_prep.copy()
fj_data_prep2 = fj_data_prep2.melt(id_vars = ['date']
                  ,value_vars=['arrivals-resident','arrivals-visitor','arrivals-total',
                                 'departures-resident','departures-visitors','departures-total',
                                 'total-in-transit','passenger-movement']
                  ,var_name='name'
                  ,value_name='value'
                  )
# fj_data_prep2 #dev check

,date,name,value
0,2018-01-01,arrivals-resident,19324.0
1,2018-02-01,arrivals-resident,9650.0
2,2018-03-01,arrivals-resident,9676.0
3,2018-04-01,arrivals-resident,13739.0
4,2018-05-01,arrivals-resident,12975.0
...,...,...,...
235,2020-02-01,passenger-movement,120653.0
236,2020-03-01,passenger-movement,82592.0
237,2020-04-01,passenger-movement,3236.0
238,2020-05-01,passenger-movement,2318.0


In [24]:
# append with metadata
properties = "{'currency': ''}"
fj_data_prep2['properties'] = properties
fj_data_prep2['frequency'] = 'monthly'
fj_data_prep2['country'] = 'fj'
fj_data_prep2['category'] = 'tourism'
fj_data_prep2['source'] = 'stats-fiji'
fj_data_prep2['metric_key'] = fj_data_prep2['country']+'-'+fj_data_prep2['name']+'-'+fj_data_prep2['date'].astype('str')
fj_data_prep2 = fj_data_prep2[['metric_key','frequency','country','category','source','name','date','value','properties']]
fj_data_prep2

,metric_key,frequency,country,category,source,name,date,value,properties
0,fj-arrivals-resident-2018-01-01,monthly,fj,tourism,stats-fiji,arrivals-resident,2018-01-01,19324.0,{'currency': ''}
1,fj-arrivals-resident-2018-02-01,monthly,fj,tourism,stats-fiji,arrivals-resident,2018-02-01,9650.0,{'currency': ''}
2,fj-arrivals-resident-2018-03-01,monthly,fj,tourism,stats-fiji,arrivals-resident,2018-03-01,9676.0,{'currency': ''}
3,fj-arrivals-resident-2018-04-01,monthly,fj,tourism,stats-fiji,arrivals-resident,2018-04-01,13739.0,{'currency': ''}
4,fj-arrivals-resident-2018-05-01,monthly,fj,tourism,stats-fiji,arrivals-resident,2018-05-01,12975.0,{'currency': ''}
...,...,...,...,...,...,...,...,...,...
235,fj-passenger-movement-2020-02-01,monthly,fj,tourism,stats-fiji,passenger-movement,2020-02-01,120653.0,{'currency': ''}
236,fj-passenger-movement-2020-03-01,monthly,fj,tourism,stats-fiji,passenger-movement,2020-03-01,82592.0,{'currency': ''}
237,fj-passenger-movement-2020-04-01,monthly,fj,tourism,stats-fiji,passenger-movement,2020-04-01,3236.0,{'currency': ''}
238,fj-passenger-movement-2020-05-01,monthly,fj,tourism,stats-fiji,passenger-movement,2020-05-01,2318.0,{'currency': ''}


In [26]:
# check and cast types
fj_data_prep2.info()
# all good :)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   metric_key  240 non-null    object        
 1   frequency   240 non-null    object        
 2   country     240 non-null    object        
 3   category    240 non-null    object        
 4   source      240 non-null    object        
 5   name        240 non-null    object        
 6   date        240 non-null    datetime64[ns]
 7   value       240 non-null    float64       
 8   properties  240 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 17.0+ KB


# Cook Islands CK
**Source:** [Cook Islands Ministry of Finance & Economic Management, Tourism and Migration Statistics](http://www.mfem.gov.ck/statistics/social-statistics/tourism-and-migration)

Notes on Usage
- Downloaded the `Mig_Statistics_Tables_
- Referred to `Table 1` (Total Arrivals and Departures) from row 519 down (Jan 2018 to Mar 2020)
   - It is missing 3 months of data relative to FJ
   - It is missing `total-in-transit` and `passenger-movement` 
      - `total-in-transit` can be manually calculated. 
- Monthly frequency
- Non seasonally adjusted

In [25]:
# evaluate dataset 1 - visitor arrival table
# source - https://www.statsfiji.gov.fj/index.php/component/advlisting/?view=download&format=raw&fileId=2148
# this was quite manual
ck_data = pd.read_excel('../data/raw/ck/Mig_Statistics_Tables_202003.xlsx'
                          ,sheet_name='Table 1'
                          ,usecols='A,D:E,H:I,L:M'
                          ,skiprows=519
                          ,nrows=29
                          ,header=None
                          ,names=['month','arrivals-total','departures-total','arrivals-visitor'
                                  ,'departures-visitors','arrivals-resident','departures-resident'])
ck_data

,month,arrivals-total,departures-total,arrivals-visitor,departures-visitors,arrivals-resident,departures-resident
0,Jan(p),11093.0,14823.0,9527.0,13405.0,1566.0,1418.0
1,Feb(p),9556.0,9809.0,8459.0,8938.0,1097.0,871.0
2,Mar(p),13155.0,12419.0,11963.0,11374.0,1192.0,1045.0
3,Apr(p),14691.0,14686.0,13231.0,13634.0,1460.0,1052.0
4,May(p),16108.0,15048.0,15171.0,14087.0,937.0,961.0
5,Jun(p),17015.0,16491.0,16083.0,15607.0,932.0,884.0
6,Jul(p),19383.0,18725.0,18332.0,17803.0,1051.0,922.0
7,Aug(p),17913.0,18545.0,16777.0,17375.0,1136.0,1170.0
8,Sep(p),17479.0,17160.0,16499.0,16196.0,980.0,964.0
9,Oct(p),16656.0,17759.0,15468.0,16684.0,1188.0,1075.0


##### proposed prep steps
- remove nulls
- manually assign years
- calculate `total-in-transit`
- cast year and month as date
- melt
- append with metadata

In [31]:
# remove nulls
ck_data_prep = ck_data.copy()
ck_data_prep.dropna(axis=0,subset=['arrivals-total'],inplace=True)
ck_data_prep.reset_index(inplace=True,drop=True)
display(ck_data_prep.shape,ck_data.shape)
# all good

(27, 7)

(29, 7)

In [33]:
# manually assign years
ck_data_prep['year'] = ''
ck_data_prep.loc[0:12,'year']='2018'
ck_data_prep.loc[12:24,'year']='2019'
ck_data_prep.loc[24:,'year']='2020'

In [36]:
# calculate `total-in-transit`
ck_data_prep['total-in-transit'] = ck_data_prep['arrivals-total'] + ck_data_prep['departures-total']

In [42]:
# cast year and month as date
ck_data_prep['year_month'] = ck_data_prep.apply(lambda x: x['year']+' '+x['month'][:3], axis=1)
ck_data_prep['date'] = pd.to_datetime(ck_data_prep['year_month'],format='%Y %b')
ck_data_prep.head()

,month,arrivals-total,departures-total,arrivals-visitor,departures-visitors,arrivals-resident,departures-resident,year,total-in-transit,year_month,date
0,Jan(p),11093.0,14823.0,9527.0,13405.0,1566.0,1418.0,2018,25916.0,2018 Jan,2018-01-01
1,Feb(p),9556.0,9809.0,8459.0,8938.0,1097.0,871.0,2018,19365.0,2018 Feb,2018-02-01
2,Mar(p),13155.0,12419.0,11963.0,11374.0,1192.0,1045.0,2018,25574.0,2018 Mar,2018-03-01
3,Apr(p),14691.0,14686.0,13231.0,13634.0,1460.0,1052.0,2018,29377.0,2018 Apr,2018-04-01
4,May(p),16108.0,15048.0,15171.0,14087.0,937.0,961.0,2018,31156.0,2018 May,2018-05-01


In [44]:
# melt
ck_data_prep2 = ck_data_prep.copy()
ck_data_prep2 = ck_data_prep2.melt(id_vars = ['date']
                  ,value_vars=['arrivals-resident','arrivals-visitor','arrivals-total',
                                 'departures-resident','departures-visitors','departures-total',
                                 'total-in-transit']
                  ,var_name='name'
                  ,value_name='value'
                  )
# ck_data_prep2 #dev check

,date,name,value
0,2018-01-01,arrivals-resident,1566.0
1,2018-02-01,arrivals-resident,1097.0
2,2018-03-01,arrivals-resident,1192.0
3,2018-04-01,arrivals-resident,1460.0
4,2018-05-01,arrivals-resident,937.0
...,...,...,...
184,2019-11-01,total-in-transit,29617.0
185,2019-12-01,total-in-transit,28835.0
186,2020-01-01,total-in-transit,27614.0
187,2020-02-01,total-in-transit,20383.0


In [45]:
# append with metadata
properties = "{'currency': ''}"
ck_data_prep2['properties'] = properties
ck_data_prep2['frequency'] = 'monthly'
ck_data_prep2['country'] = 'ck'
ck_data_prep2['category'] = 'tourism'
ck_data_prep2['source'] = 'mfem-ck'
ck_data_prep2['metric_key'] = ck_data_prep2['country']+'-'+ck_data_prep2['name']+'-'+ck_data_prep2['date'].astype('str')
ck_data_prep2 = ck_data_prep2[['metric_key','frequency','country','category','source','name','date','value','properties']]
ck_data_prep2

,metric_key,frequency,country,category,source,name,date,value,properties
0,ck-arrivals-resident-2018-01-01,monthly,ck,tourism,mfem-ck,arrivals-resident,2018-01-01,1566.0,{'currency': ''}
1,ck-arrivals-resident-2018-02-01,monthly,ck,tourism,mfem-ck,arrivals-resident,2018-02-01,1097.0,{'currency': ''}
2,ck-arrivals-resident-2018-03-01,monthly,ck,tourism,mfem-ck,arrivals-resident,2018-03-01,1192.0,{'currency': ''}
3,ck-arrivals-resident-2018-04-01,monthly,ck,tourism,mfem-ck,arrivals-resident,2018-04-01,1460.0,{'currency': ''}
4,ck-arrivals-resident-2018-05-01,monthly,ck,tourism,mfem-ck,arrivals-resident,2018-05-01,937.0,{'currency': ''}
...,...,...,...,...,...,...,...,...,...
184,ck-total-in-transit-2019-11-01,monthly,ck,tourism,mfem-ck,total-in-transit,2019-11-01,29617.0,{'currency': ''}
185,ck-total-in-transit-2019-12-01,monthly,ck,tourism,mfem-ck,total-in-transit,2019-12-01,28835.0,{'currency': ''}
186,ck-total-in-transit-2020-01-01,monthly,ck,tourism,mfem-ck,total-in-transit,2020-01-01,27614.0,{'currency': ''}
187,ck-total-in-transit-2020-02-01,monthly,ck,tourism,mfem-ck,total-in-transit,2020-02-01,20383.0,{'currency': ''}


In [46]:
# check and cast types
ck_data_prep2.info()
# all good :)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   metric_key  189 non-null    object        
 1   frequency   189 non-null    object        
 2   country     189 non-null    object        
 3   category    189 non-null    object        
 4   source      189 non-null    object        
 5   name        189 non-null    object        
 6   date        189 non-null    datetime64[ns]
 7   value       189 non-null    float64       
 8   properties  189 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 13.4+ KB


# Solomon Islands (SB)
**Source:** [Solomon Islands National Statistics Office, Visitor Arrivals Statistics First Quarter 2020 (Accessed Sept 2020](https://www.statistics.gov.sb/images/SolomonFiles/Social-and-Demography-Statistics/Visitors/2020/Visitor_Arrivals_Statistics_1st-Quarter_2020.pdf)

Handling Notes:
- Made the news - "Tourist Arrivals Fall 48.6%" - [link](https://www.solomontimes.com/news/tourist-arrivals-fall-486-due-to-covid19/9898)
- Data is quarterly
- Data has the following limitations
  - Only goes up to March 2020
  - Only contains arrivals
  - Splits resident and intended resident- sum to resident for purpose of reporting. 
- PDF
  - copied to clipboard and cleaned in a text file, named `sb_visitors.txt`

In [88]:
# sb_data = pd.read_clipboard(sep=' ',header=None) # read clipboard AFTER manually correcting spacing in a text file >_< 
sb_data = pd.read_csv('../data/raw/sb_visitors.txt',sep=' ',header=None) # refactored to read from text directly
sb_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,2016,1,"2,651","1,654","4,305",5,5,10,"4,315",301,101,402,"3,603","1,434","5,037","6,560","3,194","9,754"
1,2016,2,"3,547","1,945","5,492",11,7,18,"5,510",291,66,357,"3,917","1,762","5,679","7,766","3,780","11,546"
2,2016,3,"2,977","1,845","4,822",6,4,10,"4,832",230,83,313,"3,892","1,773","5,665","7,105","3,705","10,810"
3,2016,4,"4,685","2,933","7,618",19,4,23,"7,641",142,42,184,"4,449","2,364","6,813","9,295","5,343","14,638"
4,2017,1,"3,183","1,711","4,894",3,3,6,"4,900",199,47,246,"3,485","1,396","4,881","6,870","3,157","10,027"
5,2017,2,"4,237","2,162","6,399",3,2,5,"6,404",79,33,112,"4,314","2,111","6,425","8,633","4,308","12,941"
6,2017,3,"4,061","2,115","6,176",4,2,6,"6,182",130,46,176,"4,905","2,376","7,281","9,100","4,539","13,639"
7,2017,4,"5,402","3,354","8,756",117,49,166,"8,922",131,41,172,"4,665","2,457","7,122","10,315","5,901","16,216"
8,2018,1,"3,440","1,707","5,147",7,9,16,"5,163",91,43,134,"4,464","1,832","6,296","8,002","3,591","11,593"
9,2018,2,"5,128","2,400","7,528",2,0,2,"7,530",67,30,97,"4,644","2,377","7,021","9,841","4,807","14,648"


##### proposed prep steps
Where to start...
- name and/or drop columns
- cast year and quarter as date
- apply assumption in calculating full arrivals-resident
- melt
- append with metadata

In [89]:
# name and drop columns
sb_data_prep = sb_data.loc[:,[0,1,8,11,14,17]]

sb_data_prep.columns = ['year','quarter','arrivals-resident-1','arrivals-resident-2'
                       ,'arrivals-visitor','arrivals-total']

sb_data_prep

,year,quarter,arrivals-resident-1,arrivals-resident-2,arrivals-visitor,arrivals-total
0,2016,1,"4,315",402,"5,037","9,754"
1,2016,2,"5,510",357,"5,679","11,546"
2,2016,3,"4,832",313,"5,665","10,810"
3,2016,4,"7,641",184,"6,813","14,638"
4,2017,1,"4,900",246,"4,881","10,027"
5,2017,2,"6,404",112,"6,425","12,941"
6,2017,3,"6,182",176,"7,281","13,639"
7,2017,4,"8,922",172,"7,122","16,216"
8,2018,1,"5,163",134,"6,296","11,593"
9,2018,2,"7,530",97,"7,021","14,648"


In [90]:
# cast year and quarter as date
def quarter_mapping(q):
    mapping = {
        1: 'Jan'
        ,2: 'Apr'
        ,3: 'Jul'
        ,4:'Oct'
    }
    return mapping.get(q)
sb_data_prep['month'] = sb_data_prep.quarter.apply(lambda x: quarter_mapping(x))
sb_data_prep['year_month'] = sb_data_prep['year'].astype('str') +' '+ sb_data_prep['month']
sb_data_prep['date'] = pd.to_datetime(sb_data_prep['year_month'],format='%Y %b')
sb_data_prep

,year,quarter,arrivals-resident-1,arrivals-resident-2,arrivals-visitor,arrivals-total,month,year_month,date
0,2016,1,"4,315",402,"5,037","9,754",Jan,2016 Jan,2016-01-01
1,2016,2,"5,510",357,"5,679","11,546",Apr,2016 Apr,2016-04-01
2,2016,3,"4,832",313,"5,665","10,810",Jul,2016 Jul,2016-07-01
3,2016,4,"7,641",184,"6,813","14,638",Oct,2016 Oct,2016-10-01
4,2017,1,"4,900",246,"4,881","10,027",Jan,2017 Jan,2017-01-01
5,2017,2,"6,404",112,"6,425","12,941",Apr,2017 Apr,2017-04-01
6,2017,3,"6,182",176,"7,281","13,639",Jul,2017 Jul,2017-07-01
7,2017,4,"8,922",172,"7,122","16,216",Oct,2017 Oct,2017-10-01
8,2018,1,"5,163",134,"6,296","11,593",Jan,2018 Jan,2018-01-01
9,2018,2,"7,530",97,"7,021","14,648",Apr,2018 Apr,2018-04-01


In [93]:
# apply assumption in calculating full arrivals-resident
sb_data_prep_2 = sb_data_prep.copy()
for c in ['arrivals-resident-1','arrivals-resident-2','arrivals-visitor','arrivals-total']:
    sb_data_prep_2[c] = pd.to_numeric(sb_data_prep_2[c].astype(str).str.replace(',',''), errors='coerce')

sb_data_prep_2['arrivals-resident'] = sb_data_prep_2['arrivals-resident-1'] + sb_data_prep_2['arrivals-resident-2']
sb_data_prep_2

,year,quarter,arrivals-resident-1,arrivals-resident-2,arrivals-visitor,arrivals-total,month,year_month,date,arrivals-resident
0,2016,1,4315,402,5037,9754,Jan,2016 Jan,2016-01-01,4717
1,2016,2,5510,357,5679,11546,Apr,2016 Apr,2016-04-01,5867
2,2016,3,4832,313,5665,10810,Jul,2016 Jul,2016-07-01,5145
3,2016,4,7641,184,6813,14638,Oct,2016 Oct,2016-10-01,7825
4,2017,1,4900,246,4881,10027,Jan,2017 Jan,2017-01-01,5146
5,2017,2,6404,112,6425,12941,Apr,2017 Apr,2017-04-01,6516
6,2017,3,6182,176,7281,13639,Jul,2017 Jul,2017-07-01,6358
7,2017,4,8922,172,7122,16216,Oct,2017 Oct,2017-10-01,9094
8,2018,1,5163,134,6296,11593,Jan,2018 Jan,2018-01-01,5297
9,2018,2,7530,97,7021,14648,Apr,2018 Apr,2018-04-01,7627


In [95]:
# melt
sb_data_prep3 = sb_data_prep_2.copy()
sb_data_prep3 = sb_data_prep3.melt(id_vars = ['date']
                  ,value_vars=['arrivals-resident','arrivals-visitor','arrivals-total']
                  ,var_name='name'
                  ,value_name='value'
                  )
# sb_data_prep3.head() #dev check

,date,name,value
0,2016-01-01,arrivals-resident,4717
1,2016-04-01,arrivals-resident,5867
2,2016-07-01,arrivals-resident,5145
3,2016-10-01,arrivals-resident,7825
4,2017-01-01,arrivals-resident,5146


In [98]:
# append with metadata
properties = "{'currency': ''}"
sb_data_prep3['properties'] = properties
sb_data_prep3['frequency'] = 'quarterly'
sb_data_prep3['country'] = 'sb'
sb_data_prep3['category'] = 'tourism'
sb_data_prep3['source'] = 'stats-sb'
sb_data_prep3['metric_key'] = sb_data_prep3['country']+'-'+sb_data_prep3['frequency']+'-'+sb_data_prep3['name']+'-'+sb_data_prep3['date'].astype('str')
sb_data_prep3 = sb_data_prep3[['metric_key','frequency','country','category','source','name','date','value','properties']]
sb_data_prep3.sample(10)

,metric_key,frequency,country,category,source,name,date,value,properties
50,sb-quarterly-arrivals-total-2020-01-01,quarterly,sb,tourism,stats-sb,arrivals-total,2020-01-01,8340,{'currency': ''}
9,sb-quarterly-arrivals-resident-2018-04-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2018-04-01,7627,{'currency': ''}
22,sb-quarterly-arrivals-visitor-2017-04-01,quarterly,sb,tourism,stats-sb,arrivals-visitor,2017-04-01,6425,{'currency': ''}
42,sb-quarterly-arrivals-total-2018-01-01,quarterly,sb,tourism,stats-sb,arrivals-total,2018-01-01,11593,{'currency': ''}
45,sb-quarterly-arrivals-total-2018-10-01,quarterly,sb,tourism,stats-sb,arrivals-total,2018-10-01,15385,{'currency': ''}
30,sb-quarterly-arrivals-visitor-2019-04-01,quarterly,sb,tourism,stats-sb,arrivals-visitor,2019-04-01,7063,{'currency': ''}
13,sb-quarterly-arrivals-resident-2019-04-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2019-04-01,6684,{'currency': ''}
2,sb-quarterly-arrivals-resident-2016-07-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-07-01,5145,{'currency': ''}
27,sb-quarterly-arrivals-visitor-2018-07-01,quarterly,sb,tourism,stats-sb,arrivals-visitor,2018-07-01,7838,{'currency': ''}
3,sb-quarterly-arrivals-resident-2016-10-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-10-01,7825,{'currency': ''}


In [106]:
# one last check of info
sb_data_prep3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   metric_key  51 non-null     object        
 1   frequency   51 non-null     object        
 2   country     51 non-null     object        
 3   category    51 non-null     object        
 4   source      51 non-null     object        
 5   name        51 non-null     object        
 6   date        51 non-null     datetime64[ns]
 7   value       51 non-null     int64         
 8   properties  51 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 3.7+ KB


In [107]:
sb_data_prep3

,metric_key,frequency,country,category,source,name,date,value,properties
0,sb-quarterly-arrivals-resident-2016-01-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-01-01,4717,{'currency': ''}
1,sb-quarterly-arrivals-resident-2016-04-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-04-01,5867,{'currency': ''}
2,sb-quarterly-arrivals-resident-2016-07-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-07-01,5145,{'currency': ''}
3,sb-quarterly-arrivals-resident-2016-10-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-10-01,7825,{'currency': ''}
4,sb-quarterly-arrivals-resident-2017-01-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2017-01-01,5146,{'currency': ''}
5,sb-quarterly-arrivals-resident-2017-04-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2017-04-01,6516,{'currency': ''}
6,sb-quarterly-arrivals-resident-2017-07-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2017-07-01,6358,{'currency': ''}
7,sb-quarterly-arrivals-resident-2017-10-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2017-10-01,9094,{'currency': ''}
8,sb-quarterly-arrivals-resident-2018-01-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2018-01-01,5297,{'currency': ''}
9,sb-quarterly-arrivals-resident-2018-04-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2018-04-01,7627,{'currency': ''}


# Writing to DB

In [99]:
try:
    engine = sqlalchemy.create_engine(connection_str)
    conn = engine.connect()
except:
    print('Database connection error - check creds')

In [110]:
# confirm tables are present as expected
engine.table_names() 

['test', 'test2', 'country_metrics', 'stg_country_metrics']

In [101]:
data_write = pd.concat([sb_data_prep3,ck_data_prep2,fj_data_prep2])
data_write.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 239
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   metric_key  480 non-null    object        
 1   frequency   480 non-null    object        
 2   country     480 non-null    object        
 3   category    480 non-null    object        
 4   source      480 non-null    object        
 5   name        480 non-null    object        
 6   date        480 non-null    datetime64[ns]
 7   value       480 non-null    float64       
 8   properties  480 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 37.5+ KB


In [102]:
data_write.head()

,metric_key,frequency,country,category,source,name,date,value,properties
0,sb-quarterly-arrivals-resident-2016-01-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-01-01,4717.0,{'currency': ''}
1,sb-quarterly-arrivals-resident-2016-04-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-04-01,5867.0,{'currency': ''}
2,sb-quarterly-arrivals-resident-2016-07-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-07-01,5145.0,{'currency': ''}
3,sb-quarterly-arrivals-resident-2016-10-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-10-01,7825.0,{'currency': ''}
4,sb-quarterly-arrivals-resident-2017-01-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2017-01-01,5146.0,{'currency': ''}


In [105]:
# check db pre-write
pd.read_sql("""
select country, category, name, count(date) as count_measure
from public.country_metrics
group by 1,2,3 order by 1,2,3

""",con= engine)

,country,category,name,count_measure
0,ck,tourism,arrivals-resident,27
1,ck,tourism,arrivals-total,27
2,ck,tourism,arrivals-visitor,27
3,ck,tourism,departures-resident,27
4,ck,tourism,departures-total,27
5,ck,tourism,departures-visitors,27
6,ck,tourism,total-in-transit,27
7,ck,trade,exports-fob-domestic,40
8,ck,trade,exports-fob-reexport,40
9,ck,trade,exports-fob-total,40


#### estimate change
- new country value - sb (sl is likely a typo)
- new country category - tourism
- 3 new measures for sb
- 17 new values 

In [109]:
# paranoid copying of table prior to writing

with engine.connect() as con:
    con.execute("""
    drop table if exists public.stg_country_metrics
    ;
    create table public.stg_country_metrics as 
    select * from public.country_metrics
    ;;
    """)

In [117]:
data_sql = pd.read_sql("select * from public.country_metrics where category = 'tourism'",con=engine)
data_write_new = pd.concat((data_write, data_sql)).drop_duplicates(subset=['metric_key'], keep=False)
data_write_new

,metric_key,frequency,country,category,source,name,date,value,properties
0,sb-quarterly-arrivals-resident-2016-01-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-01-01,4717.0,{'currency': ''}
1,sb-quarterly-arrivals-resident-2016-04-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-04-01,5867.0,{'currency': ''}
2,sb-quarterly-arrivals-resident-2016-07-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-07-01,5145.0,{'currency': ''}
3,sb-quarterly-arrivals-resident-2016-10-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-10-01,7825.0,{'currency': ''}
4,sb-quarterly-arrivals-resident-2017-01-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2017-01-01,5146.0,{'currency': ''}
5,sb-quarterly-arrivals-resident-2017-04-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2017-04-01,6516.0,{'currency': ''}
6,sb-quarterly-arrivals-resident-2017-07-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2017-07-01,6358.0,{'currency': ''}
7,sb-quarterly-arrivals-resident-2017-10-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2017-10-01,9094.0,{'currency': ''}
8,sb-quarterly-arrivals-resident-2018-01-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2018-01-01,5297.0,{'currency': ''}
9,sb-quarterly-arrivals-resident-2018-04-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2018-04-01,7627.0,{'currency': ''}


In [122]:
data_write_new.to_sql('country_metrics'
                 ,con = engine
                 ,if_exists='append'
                 ,index=False
                 ,method='multi')

In [123]:
# tests
pd.read_sql("""
select country, category, name, count(date) as count_measure
from public.country_metrics
group by 1,2,3 order by 1,2,3

""",con= engine)

,country,category,name,count_measure
0,ck,tourism,arrivals-resident,27
1,ck,tourism,arrivals-total,27
2,ck,tourism,arrivals-visitor,27
3,ck,tourism,departures-resident,27
4,ck,tourism,departures-total,27
5,ck,tourism,departures-visitors,27
6,ck,tourism,total-in-transit,27
7,ck,trade,exports-fob-domestic,40
8,ck,trade,exports-fob-reexport,40
9,ck,trade,exports-fob-total,40


In [120]:
# if you've done an unsuccessful write

# with engine.connect() as con:
#     con.execute("""
#     drop table if exists public.country_metrics
#     ;
#     create table public.country_metrics as 
#     select * from public.stg_country_metrics
#     ;;
#     """)